# Load data

In [18]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# folder should be named 'nb_stations'_'nb_vans'_'S'_*
def load_results(data_folder):
    results_folder = os.path.join("../../results/data", data_folder)
    nb_stations = int(data_folder.split('_')[2])
    nb_vans = int(data_folder.split('_')[4])

    with open(os.path.join(results_folder, 'x.csv'), 'r') as f:
        x = f.read().splitlines()
    x = np.array(x).astype(float).reshape((nb_stations,nb_stations,nb_vans,24),order='F')

    with open(os.path.join(results_folder, 'v.csv'), 'r') as f:
        v = f.read().splitlines()
    v = np.array(v).astype(float).reshape((nb_stations,nb_stations,nb_vans,24),order='F')

    with open(os.path.join(results_folder, 'z.csv'), 'r') as f:
        z = f.read().splitlines()
    z = np.array(z).astype(float).reshape((nb_stations,nb_stations,nb_vans,24),order='F')

    with open(os.path.join(results_folder, 'y.csv'), 'r') as f:
        y = f.read().splitlines()
    y = np.array(y).astype(float).reshape((nb_stations,24),order='F')

    with open(os.path.join(results_folder, 'w.csv'), 'r') as f:
        w = f.read().splitlines()
    w = np.array(w).astype(float).reshape((nb_stations,nb_stations,24),order='F')

    with open(os.path.join(results_folder, 'u.csv'), 'r') as f:
        u = f.read().splitlines()
    u = np.array(u).astype(float).reshape((nb_stations,nb_stations,24),order='F')

    with open(os.path.join(results_folder, 'd.csv'), 'r') as f:
        d = f.read().splitlines()
    d = np.array(d).astype(float).reshape((nb_stations,nb_stations,24),order='F')

    return x, v, z, y, w, u, d

In [19]:
def plot_station(station_id, results_folder):
    _, _, z, _, w, _, d = load_results(results_folder)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(24), y=w[station_id,:,:].reshape(-1, w[station_id,:,:].shape[-1]).sum(axis=0), mode='lines', name='Actual Trips'))
    fig.add_trace(go.Scatter(x=np.arange(24), y=d[station_id,:,:].reshape(-1, d[station_id,:,:].shape[-1]).sum(axis=0), mode='lines', name='Demand'))
    #fig.add_trace(go.Scatter(x=np.arange(24), y=y[station_id,:], mode='lines', name='Inventory'))
    # add vertical lines when rebalancing is done
    for i in range(0,24):
        if z[:,station_id,:,i].sum() > 0:
            fig.add_vline(x=i, line_width=1, line_dash="dash", line_color="Black", annotation_text=f"+{int(z[:,station_id,:,i].sum())} bikes")
    fig.update_layout(title='Station ' + str(station_id), xaxis_title='Time', yaxis_title='Number of trips & demand', width=800, height=400)
    fig.show()

def plot_all_stations(simulation_folder, results_folder):
    pass

In [63]:
def plot_station_with_without(station_id, results_folder_with, results_folder_without):
    _, _, z, y, w, _, d = load_results(results_folder_with)
    _, _, _, _, w_, _, _ = load_results(results_folder_without)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(24), y=w[station_id,:,:].reshape(-1, w[station_id,:,:].shape[-1]).sum(axis=0), mode='lines', name='Actual Trips'))
    fig.add_trace(go.Scatter(x=np.arange(24), y=w_[station_id,:,:].reshape(-1, w_[station_id,:,:].shape[-1]).sum(axis=0), mode='lines', name='Actual Trips without rebalancing'))
    fig.add_trace(go.Scatter(x=np.arange(24), y=d[station_id,:,:].reshape(-1, d[station_id,:,:].shape[-1]).sum(axis=0), mode='lines', name='Demand'))
    fig.add_trace(go.Scatter(x=np.arange(24), y=y[station_id,:], mode='lines', name='Inventory'))
    #add vertical lines when rebalancing is done
    for i in range(0,24):
        if z[:,station_id,:,i].sum() > 0.5:
            fig.add_vline(x=i-1, line_width=1, line_dash="dash", line_color="Black", annotation_text=f"+{int(z[:,station_id,:,i].sum())}")
        if z[station_id,:,:,i].sum() > 0.5:
            fig.add_vline(x=i-1, line_width=1, line_dash="dash", line_color="sienna", annotation_text=f"-{int(z[station_id,:,:,i].sum())}")
    fig.update_layout(title='Station ' + str(station_id), xaxis_title='Time', yaxis_title='Number of trips & demand', width=800, height=400)
    fig.show()

In [64]:
plot_station(0, "2022-12-07T133257_nstations_31_nvans_2_capacity_25_lambda_15_timelimit_120")

In [65]:
plot_station_with_without(9, "2022-12-08T014608_nstations_31_nvans_5_capacity_20_lambda_4_timelimit_160", "2022-12-08T014611_nstations_31_nvans_5_capacity_20_lambda_4_timelimit_160_norebal")